In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from ipywidgets import Textarea
from iam_utils import *

import tabulate
from ipywidgets import Label, Button, Output, FloatSlider, HBox, VBox, Layout, HTML, Accordion
from appmode_functions import get_recompute, check_user_value, get_user_value

from widget_code_input import WidgetCodeInput

In [ ]:
import scipy as sc
import numpy.linalg as npl
from numpy.linalg import norm
import scipy.linalg as scl
from scipy.spatial.distance import pdist,squareform
import ase
from tqdm import tqdm_notebook as tqdm_cs

In [ ]:
from ipywidgets import interactive,FloatSlider,interact
import ipywidgets as widgets

In [ ]:
#### AVOID folding of output cell 

In [ ]:
%%html

<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:4000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

# Lattice dynamics of a one-dimensional crystal 

Consider a one-dimensional crystal with lattice parameter $a$, so that the positions of atoms in the ideal lattice are $r_n=na$. Each atom may be displaced by a finite amount $u_n$ from the ideal position, so that the actual atomic positions are $r_n=na+u_n$.

<img src="figures/TBC-linear-chain.png" width="500" height="250" />

The energy can be computed in terms of a Taylor expansion: if $na$ are equilibrium lattice positions, $\partial E/\partial u_n = 0$ and so the lowest-order term involves the matrix of second derivatives,
$$
E(\mathbf{u}) \approx E_0 + \frac{1}{2} \sum_{ij} \left.\frac{\partial^2 E}{\partial u_i \partial u_j}\right|_{\bu=0} u_i u_j
$$

The matrix of second derivatives $D_{ij} \equiv \left.{\partial^2 E}/{\partial u_i \partial u_j}\right|_{\bu=0}$ is usually called the *matrix of force constants*, and defines the response of the crystal to perturbations of the atomic positions. 

## Exercise 1: harmonic energy and forces

<i style="color:blue"> 
The force acting on the atoms when they are displaced from their equilibrium position is given by $\mbf{f}=-\partial E/\partial \bu$. Write down the expression in terms of $\mbf{D}$ and $\bu$.
Write a function that computes the energy and the forces for a lattice containing two atoms, with displacements $u_0$ and $u_1$. Check that the function gives the correct results by validating your input
</i>    

In [ ]:
def draw_forces(ax, D00, D01, D10, D11, u0, u1):
    ax.add_artist(plt.Circle((-1+u0,0), 0.1, color='red'))
    ax.add_artist(plt.Circle((1+u1,0), 0.1, color='red'))
    ax.set_xlim(-1.5,1.5)
    ax.set_ylim(-1.5,1.5)
    ax.plot([0,1],[0,0], 'g')
    rea

In [ ]:
p = WidgetPlot(draw_forces, WidgetParbox(D00 = (1.0, 0, 10, 0.1, r'$D_{00}$'),
                                         D01 = (0.0, 0, 10, 0.1, r'$D_{01}$'),
                                         D10 = (0.0, 0, 10, 0.1, r'$D_{10}$'),
                                         D11 = (4.0, 0, 10, 0.1, r'$D_{11}$'),
                                         u0 = (1.0, -4, 4, 0.1, r'$u_0$'),
                                         u1 = (-0.5, -4, 4, 0.1, r'$u_0$'),
                                         ));

In [ ]:
display(p)

In [ ]:
class SetUpLattice2(object):
    def __init__(self):        
        self.plot_box = Output()
        with self.plot_box:
            self.the_figure, self.the_plot = plt.subplots(figsize=(5,5))
        
        # set up the value checker
        self.check_function_output = Output()
        self.check_accordion = Accordion(children=[self.check_function_output], selected_index=None)
        self.check_accordion.set_title(0, 'Check the value of your function')
        
        # set up the sliders
        D_00_widget = FloatSlider(
            value=1, min=-2, max=2,step=0.1,
            description=r'\(D_{00}\)',
            continuous_update=False, 
            style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))
        D_01_widget = FloatSlider(
            value=-1, min=-2, max=2,step=0.1,
            description=r'\(D_{01}\)',
            continuous_update=False, 
            style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))
        D_10_widget = FloatSlider(
            value=-1, min=-2, max=2,step=0.1,
            description=r'\(D_{10}\)',
            continuous_update=False, 
            style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))
        D_11_widget = FloatSlider(
            value=1, min=-2, max=2,step=0.1,
            description=r'\(D_{11}\)',
            continuous_update=False, 
            style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))
        u_0_widget = FloatSlider(
            value=-0.1, min=-0.2, max=0.2,step=0.01,
            description=r'\(u_{0}\)',
            continuous_update=False, 
            style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))
        u_1_widget = FloatSlider(
            value=0.05, min=-0.2, max=0.2,step=0.01,
            description=r'\(u_{1}\)',
            continuous_update=False, 
            style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))     
        self.sliders_dict = dict(D_00=D_00_widget,D_01=D_01_widget,D_10=D_10_widget,D_11=D_11_widget,
                                u_0=u_0_widget,u_1=u_1_widget)
        
        # set up the test values
        self.test_values_dict = dict(D_00=[2,1],D_01=[1], D_10=[-2], D_11=[1,1], u_0=[-0.1,0.2], u_1=[0.05,-0.01])
        
        # set upt the code widget window
        self.code_widget = WidgetCodeInput(
            function_name="lattice_energy_force", 
            function_parameters="D_00, D_01, D_10, D_11, u_0, u_1",
            docstring="""
        Computes energy and force associated with a given displacement of two atoms in a harmonic lattice. 

        :param D_ij: elements of the matrix of second derivatives
        :param u_i: atomic displacements
        
        :return: A tuple containing the lattice energy and the force, ([E, f_0, f_1]])
        """,
            function_body="""
    # Write your solution, and test it by moving the sliders
e = 0.0
f = [0, 0]
return e, f[0], f[1]
    """
        )
        
        self.widgets = [slider for slider in self.sliders_dict.values()]
        self.widgets += [self.code_widget]
        
        self.input_box = VBox([slider for slider in self.sliders_dict.values()])
        # self.plot = interactive_output(self.replot, self.sliders_dict)
    
    def display(self):
        display(self.code_widget)
        display(self.check_accordion)
        display(self.input_box, self.the_figure)        
        
    def reference_func(self,  D_00, D_01, D_10, D_11, u_0, u_1):
        
        pot = 0.5*(D_00*u_0**2 + (D_10+D_01)*u_0*u_1 + D_11*u_1**2)
        force = [ -(D_00*u_0 + 0.5*(D_10+D_01)*u_1),  -(D_11*u_1 + 0.5*(D_10+D_01)*u_0) ]
        return pot, force[0], force[1]
    
    def replot(self, D_00, D_01, D_10, D_11, u_0=0, u_1=0):
        # Clean up the graph
        self.the_plot.axes.clear()

        latt0 = plt.Circle((0, 0), 0.12, color='gray')
        latt1 = plt.Circle((1, 0), 0.12, color='gray')
        part0 = plt.Circle((0+u_0, 0), 0.1, color='red')
        part1 = plt.Circle((1+u_1, 0), 0.1, color='red')
        
        self.the_plot.add_artist(latt0)        
        self.the_plot.add_artist(latt1)
        self.the_plot.add_artist(part0)
        self.the_plot.add_artist(part1)        
        
        # computes energy & forces
        U, f0, f1 = self.reference_func(D_00, D_01, D_10, D_11, u_0, u_1)
        
        if f0**2>1e-5:
            arr0 = mpl.patches.FancyArrow(0+u_0,0,f0,0,width=0.02)
            self.the_plot.add_artist(arr0) 
        if f1**2>1e-5:
            arr1 = mpl.patches.FancyArrow(1+u_1,0,f1,0,width=0.02)
            self.the_plot.add_artist(arr1) 
        
        ## (Try to) plot user value
        user_value = None
        try:
            user_value = get_user_value()
        except Exception:
            # Just a guard not to break the visualization, we should not end up here
            pass 
        
        try:
            if user_value is not None:
                self.the_plot.plot([user_value], [0], 'or')    
        except Exception:
            # We might end up here if the function does not return a float value
            pass 

        self.the_plot.set_xlim(-0.5,1.5)
        self.the_plot.set_xlabel("x/a")
        self.the_plot.set_ylim(-1,1)
        self.the_plot.set_yticks([])
        
        # Redraw
        self.the_figure.canvas.draw()
        self.the_figure.canvas.flush_events()

In [ ]:
setup_class1 = SetUpLattice2()

recompute1 = get_recompute(setup_class1)

_ = recompute1(None)

setup_class1.display()

<i style="color:blue"> 
Set both atoms to the same displacement $u_0=u_1>0$, then adjust the values of the matrix of force constants. 
    
What do you observe? Is this a physical behavior if these two atoms represent a cell in a periodic system?
</i>    

## Properties of the matrix of force constants

When the matrix of force constants describes the interactions in a periodic crystal, it must fulfill several physical conditions:
* The matrix is symmetric $D_{ij}=D_{ji}$
* The elements only depend on the separation between the atoms, $D_{ij} \equiv -K_{|i-j|}$
* A uniform displacement of the atoms leads to zero force: this leads to a condition called *acoustic sum rule* 
   $\sum_i D_{ij} = 0$

## A solution for a periodic lattice

We look for a way to express the time dependence of the lattice vibrations, $u_n(t)$. The lattice vibrations must satisfy Newton's equation, which implies that $m\ddot{u}_i=f_i = -\sum_j D_{ij} u_j$. 
To solve this in the most general way possible, we express $u_n(t)$ on a plane wave basis, 
$$
u_n(t) = \int \D \omega \D q \hat{u}(q,\omega) e^{\iu (q n a - \omega t)}.
$$
Given the periodicity of the lattice, one only needs to consider $-\pi/a < q < \pi/a$.
By substituting into the equations of motion, and noting that both the time derivative and the sum over the matrix of force constants are linear operation that commute with the integration, one sees that the only way to ensure a consistent solution for any $\hat{u}(q,\omega)$ is to have
$$
m \omega^2 e^{\iu (q i a - \omega t)} = \sum_j D_{ij} e^{\iu (q j a - \omega t)},
$$
that is, there is a *dispersion relation* that links $q$ and $\omega$, $m \omega(q)^2 = -\sum_j K_{j} e^{-\iu q j a}$.

## Exercise 2: the dispersion relation

<i style="color:blue">
Write the function that computes $m \omega(q)^2$ given the force constants $K_{1\ldots 4}$, and plot the frequencies of the lattice vibrations as a function of the wavevector. You can check your solution by comparing the value of the function you computed (red line) with a reference implementation (dashed gray line)
</i>


Hints:
* When you change the function, move the sliders to update the plot
* You will need functions from numpy, that you can access with `np.XXXX` from the function body
* The sum extends to negative values of $j$. Ask yourself what should be the value of $K_{-j}$ given the properties of the matrix of force constants.  
* What happens if you combine the terms $j$ and $-j$? Is the dispersion relation real?
* Don't forget there is also a $K_0$! Use the acoustic sum rule to determine its value

In [ ]:
class SetUpDispersion(object):
    def __init__(self):        
        self.plot_box = Output()
        with self.plot_box:
            self.the_figure, self.the_plot = plt.subplots(figsize=(5,5))
            self.the_plot.set_xlabel("x [l]")
            self.the_plot.set_xlabel(" [e]")
            
        # set up the value checker (not used here! TODO: clean up so it's not always needed)
        self.check_function_output = Output()
                        
        # set up the sliders
        K_1_widget = FloatSlider(
            value=1, min=-2, max=2,step=0.1,
            description=r'\(K_1\)',
            continuous_update=False, 
            style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))
        K_2_widget = FloatSlider(
            value=0, min=-2, max=2,step=0.1,
            description=r'\(K_2\)',
            continuous_update=False, 
            style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))
        K_3_widget = FloatSlider(
            value=0, min=-2, max=2,step=0.1,
            description=r'\(K_3\)',
            continuous_update=False, 
            style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))
        K_4_widget = FloatSlider(
            value=0, min=-2, max=2,step=0.1,
            description=r'\(K_{4}\)',
            continuous_update=False, 
            style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))
        self.sliders_dict = dict(K_1=K_1_widget,K_2=K_2_widget,K_3=K_3_widget,K_4=K_4_widget)
        
        # set up the test values
        self.test_values_dict = dict(K_1=[2,1],K_01=[1], K_10=[-2], K_11=[1,1], u_0=[-0.1,0.2], u_1=[0.05,-0.01])
        
        # set upt the code widget window
        self.code_widget = WidgetCodeInput(
            function_name="m_omega_square", 
            function_parameters="K_1, K_2, K_3, K_4, q",
            docstring="""
        Computes the dispersion relation for a one-dimensional lattice given the first four force constants

        :param K_i: force constants
        :param q: wavevector to compute the dispersion relation
        
        :return: the value of m * omega**2
        """,
            function_body="""
    # Write your solution, and test it by moving the sliders
import numpy as np
return 0.0
"""
        )
        
        self.widgets = [slider for slider in self.sliders_dict.values()]
        #self.widgets += [self.code_widget]
        
        self.input_box = VBox([slider for slider in self.sliders_dict.values()])
        # self.plot = interactive_output(self.replot, self.sliders_dict)
    
    def display(self):
        display(self.code_widget)
        display(self.input_box, self.the_figure)    
        
    def reference_func(self, K_1, K_2, K_3, K_4, q):
        
        K_0 = -2*(K_1+K_2+K_3+K_4)   # acoustic sum rule 
        
        return -(K_0 + 2*K_1 * np.cos(q*1)+ 
                      2*K_2 * np.cos(q*2)+ 
                      2*K_3 * np.cos(q*3)+
                      2*K_4 * np.cos(q*4))
    
    
    def replot(self, K_1, K_2, K_3, K_4):
        # Clean up the graph
        self.the_plot.axes.clear()

        
        q = np.linspace(-np.pi, np.pi, 200)
        w2ref = self.reference_func( K_1, K_2, K_3, K_4, q)
        
        user_function = self.code_widget.get_function_object()         
        w2 = user_function(K_1, K_2, K_3, K_4, q)
        
        self.the_plot.plot(q, np.sqrt(np.abs(w2))*np.sign(w2ref), 'r', linewidth=2)
        self.the_plot.plot(q, np.sqrt(np.abs(w2ref))*np.sign(w2ref), 'k--')
        
        # plt.show()
        ## (Try to) plot user value
        user_value = None
        try:
            user_value = get_user_value()
        except Exception:
            # Just a guard not to break the visualization, we should not end up here
            pass 
        
        try:
            if user_value is not None:
                self.the_plot.plot([user_value], [0], 'or')    
        except Exception:
            # We might end up here if the function does not return a float value
            pass 

        self.the_plot.set_xlim(-np.pi, np.pi)
        self.the_plot.set_xlabel("q a")
        self.the_plot.set_ylim(-1,4)
        self.the_plot.set_ylabel(r"$\omega\sqrt{m}$")
        
        # Redraw
        self.the_figure.canvas.draw()
        self.the_figure.canvas.flush_events()

In [ ]:
type(True)

In [ ]:
setup_class2 = SetUpDispersion()

recompute2 = get_recompute(setup_class2)

_ = recompute2(None)

setup_class2.display()

<i style="color:blue"> 
Make sure the reference implementation matches your function for all values of the force constants. 
Observe what happens as you change the parameters. 
</i>    

Why does it make sense to truncate the expansion at $K_4$? How do you expect the force constants to change with increasing $j$? Hint: go back to look at the expression of the force acting on atom $0$. How does it depend on the displacement of atom $j$?

Note that the plot visualizes $\operatorname{sign}(\omega^2) \sqrt{|m\omega^2|}$, so negative values on the plot correspond to so-called *imaginary frequencies*, i.e. values of $q$ for which $\omega^2$ is negative. 

How will the energy change if you introduce a distortion with a periodicity corresponding to a $q$ associated with a negative $\omega^2$? What does this imply in terms of the stability of the crystal? 
   